In [ ]:
# Install all necessary packages
import numpy as np
import os
import csv
import pandas as pd
from cmcrameri import cm
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import OrderedDict
from matplotlib.lines import Line2D  
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io import shapereader
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.patches as mpatches
import cmocean
from scipy.interpolate import griddata

In [ ]:
### CODE FROM HERE BASED ON CODE EXAMPLES BY DR ANDREW MERDITH FROM EARTHBYTE GROUP ###
# Andrew's rotation data files 
rot_file = './Andrew_data/'
topology_features = [rot_file+'MER21/250-0_plate_boundaries_Merdith_et_al.gpml',
                     rot_file+'MER21/410-250_plate_boundaries_Merdith_et_al.gpml',
                     rot_file+'MER21/TopologyBuildingBlocks_Merdith_et_al.gpml',
                     rot_file+'MER21/1000-410-Convergence_Merdith_et_al.gpml',
                     rot_file+'MER21/1000-410-Divergence_Merdith_et_al.gpml',
                     rot_file+'MER21/1000-410-Topologies_Merdith_et_al.gpml',
                     rot_file+'MER21/1000-410-Transforms_Merdith_et_al.gpml']

In [ ]:
# Adding rotation files into collection for pygplates
static_polygons = pygplates.FeatureCollection(
    rot_file+'MER21/shapes_static_polygons_Merdith_et_al.gpml')
polygons = pygplates.FeatureCollection(
    rot_file+'MER21/Global_EarthByte_GeeK07_COB_Terranes_ContinentsOnly_MER21_edits.gpml')
rotation_model = pygplates.RotationModel(rot_file+'MER21/1000_0_rotfile_Merdith_et_al.rot')
coastlines = pygplates.FeatureCollection(
    rot_file+'MER21/shapes_continents_Merdith_et_al.gpml')

In [ ]:
# Gplately model using above files
model = gplately.PlateReconstruction(
    rotation_model, topology_features, coastlines)

In [ ]:
# Make present-day plot of points
gPlot = gplately.plot.PlotTopologies(model, coastlines=coastlines, COBs=polygons)

In [ ]:
# Read in data to be reconstructed (modern lats and lons to paleo lats and lons)
# Currently has my original data file in (i.e. before I replaced lats and lons with recon_lats and recon_lons in the excel file)
filtered_data = pd.read_csv("Toarcian_data_thisone_final_final_final.csv")
filtered_data

In [ ]:
print(filtered_data.columns)
recon_lons= 
recon_lats= 

In [ ]:
#compute geometry as points (we will turn into polygons next) and convert to geopandas
filtered_data['geometry'] = filtered_data.apply(lambda row: pygplates.PointOnSphere(float(row.lat),
                                                              float(row.lon)), axis=1)

In [ ]:
#group our entries by their index so we can find which ones are lines and not points
for ind, (name, group) in enumerate(filtered_data.groupby('Site number')): 

    #print(ind, len(group))
    #line has more than 1 entry
    if len(group) > 1:
        
        test = group
        
        polyLine = pygplates.PolylineOnSphere(group['geometry'])
        filtered_data.at[group.index[0], 'geometry'] = polyLine
        #drop second row now we no longer need it
        filtered_data.drop([group.index[1]], inplace=True)
#reset index
filtered_data.reset_index(inplace=True)

In [ ]:
pygplates.PointOnSphere(float(group.lat), float(group.lon))
filtered_data.iloc[0]

In [ ]:
# Loop through list of poles and produce GPML file of multiple VGPs.
geometry = []
plantCountry = []
plantArea = []
plantFunction = []
plantLowerAge = []
plantUpperAge = []
plantgeometry=[]
plantEnv=[]
plantID=[]

for i in np.arange(0, len(filtered_data)):
       
    geometry.append(filtered_data.geometry[i])
    if filtered_data['Area'][i] is np.nan:
        plantCountry.append(filtered_data.Area[i])
        plantFunction.append(filtered_data['Site number'][i])
    else:
        plantCountry.append((filtered_data['Area'][i], filtered_data.Area[i]))
        plantFunction.append(filtered_data['Site number'][i])
        plantEnv.append(filtered_data['Redox interpretation'][i])
        plantID.append(filtered_data['Study site'][i])
    plantLowerAge.append(int(600))
    plantUpperAge.append(int(0))
    


In [ ]:
#Create new GPlates Feature Collection
plantFeatureCollection = pygplates.FeatureCollection()


In [ ]:
#assign data to feature
for j in np.arange(0, len(filtered_data)):
    plantFeature = pygplates.Feature.create_reconstructable_feature(
        pygplates.FeatureType.gpml_unclassified_feature,
        geometry[j],
        name = str(plantID[j]),
        valid_time=(float(plantLowerAge[j]), float(plantUpperAge[j])))
    # Add newly created feature to existing Feature Collection
  #  plantFeatureCollection.add(plantFeature)
    {'Function' : str(plantFunction[j])}
    # Add newly created feature to existing Feature Collection
    plantFeatureCollection.add(plantFeature)
#note: a lot of import errors

In [ ]:
# Assign plate ids
# Create new GPlates Feature Collection
plate_partitioner = pygplates.PlatePartitioner(coastlines, rotation_model)
partitioned_plant_features = plate_partitioner.partition_features(
                                        plantFeatureCollection,
                                        partition_method = pygplates.PartitionMethod.most_overlapping_plate)

plantFeatureCollection = pygplates.FeatureCollection(partitioned_plant_features)

In [ ]:
gPlot.time = 183 
phanerozoic_data = plantFeatureCollection

reconstructed_data = model.reconstruct(phanerozoic_data, gPlot.time)

recon_lons = []
recon_lats = []
env = []
for ind, i in enumerate(reconstructed_data):
    recon_lons.append(i.get_reconstructed_geometry().to_lat_lon()[1])
    recon_lats.append(i.get_reconstructed_geometry().to_lat_lon()[0])
    env.append(i.get_feature().get_name())

all_env_codes = np.asarray(env)

In [ ]:
#export lat/lon and land code
with open("TOAE_data.csv", 'w', newline='') as file:
    writer = csv.writer(file, dialect='excel')
    writer.writerow(recon_lons)
    writer.writerow(recon_lats)
    writer.writerow(all_env_codes)

In [ ]:
# print length of reconstructed data 
len(reconstructed_data)

In [ ]:
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Tahoma"

fig = plt.figure(figsize=(18,12))
ax = fig.add_subplot(111, projection=ccrs.Robinson(central_longitude=0))

#gPlot.plot_continents(ax, facecolor='0.8')
gPlot.plot_coastlines(ax, color='0.85')
gPlot.plot_ridges_and_transforms(ax, lw=1, alpha=0.5, color='#545EB3')
gPlot.plot_trenches(ax, lw=1, color='#853a2b')
gPlot.plot_misc_boundaries(ax, lw=0.5, color='k')
gPlot.plot_subduction_teeth(ax, color='#853a2b')

ax.scatter(recon_lons, recon_lats, transform=ccrs.Geodetic())

lat_spacing = 30
lon_spacing = 30
ax.gridlines(
    draw_labels=True,
    ylocs=list(np.arange(-90, 90+lat_spacing, lat_spacing)),
    xlocs=list(np.arange(-180,180+lon_spacing, lon_spacing)),
)


### CODE BASED ON CODE EXAMPLES BY DR ANDREW MERDITH FROM EARTHBYTE GROUP ENDS HERE ###

In [ ]:
### MY ORIGINAL CODE STARTS HERE ###
# read csv
data = pd.read_csv('Map40_185Ma.csv')  

# lat, lon, elev
lons = data['# lon']  
lats = data['lat']   
elevations = data['elev']  

# make map
fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

# plot data
sc = ax.scatter(lons, lats, c=elevations, cmap='viridis', s=10, transform=ccrs.PlateCarree())

# colour bar
plt.colorbar(sc, label='Elevation (m)')

# plot
plt.show()

In [ ]:
#loading in same dataset but with elevations and recon_lats and recon_lons instead of modern lats and lons 
filtered_data = pd.read_csv("TOAE_lats_lons_elevations_final_final.csv")

In [ ]:
#read nc file
nc = NetCDFFile('./Scotese_Wright_2018_Maps_1-88_1degX1deg_PaleoDEMS_nc_v2/Map40_PALEOMAP_1deg_Early_Jurassic_185Ma.nc')

In [ ]:
#view nc file
print(nc.variables)

In [ ]:
#assign variables- note you will have to re-run this cell every time you want to adjust the map, so I put this code in the top of that code as well 
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
z = nc.variables['z'][:]

In [ ]:
# plot and save elevation map and datapoints
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
z = nc.variables['z'][:]

data_lons = filtered_data['lon']
data_lats = filtered_data['lat']

recon_lons_a=np.array(data_lons)
recon_lats_a=np.array(data_lats)

#figure size and layout 
plt.figure(figsize=(20,10))
plt.tight_layout()

#plot contour 
plt.contourf(lon, lat, z, cmap=cmocean.cm.deep_r)
colorbar= plt.colorbar()
colorbar.set_label('Elevation (m)', fontsize= 14)

#ensure axes on same scale
plt.gca().set_aspect("equal")

#plot scatter points
plt.scatter(recon_lons, recon_lats, color='red', marker='o')

#organise scatter points by oxygen state 
labels= filtered_data['Redox interpretation']
#label_colourmap = {label: color for label, color in zip(unique_labels,['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'brown', 'pink'])}
# Loop through each data point and plot based on its label
for lon, lat, label in zip(data_lons, data_lats, labels):
    if label == 'Euxinic ':
        plt.scatter(lon, lat, color='maroon', marker='o') # Plot 'Oxic' points
    elif label == 'Anoxic ':
        plt.scatter(lon, lat, color='lightcoral', marker='o')
    elif label == 'Anoxic':
        plt.scatter(lon, lat, color='lightcoral', marker='o')
    elif label == 'Suboxic ':
        plt.scatter(lon, lat, color='darkorange', marker='o')
    else:
        print('youre missing one')


#labels and title
plt.xlabel('Longitude', fontsize= 14)
plt.ylabel('Latitude', fontsize= 14)
plt.title('Map of the Toarcian period with Sample Data Points', fontsize= 16)

#legend 
Euxinic_patch = mpatches.Patch(color='maroon', label='Euxinic')
Anoxic_patch = mpatches.Patch(color='lightcoral', label='Anoxic')
Suboxic_patch = mpatches.Patch(color='darkorange', label='Suboxic')
plt.legend(handles=[Euxinic_patch, Anoxic_patch, Suboxic_patch])

plt.show 
plt.savefig('T-OAE_map_single')

In [ ]:
# extract elevations for each datapoint (manually)

def find_elevation_from_nc(recon_lats, recon_lons, lat, lon, z):
     Create a meshgrid of the NC file coordinates
    lon_mesh, lat_mesh = np.meshgrid(lon, lat)
    
# reshape coordinates and elevation data
    points = np.column_stack((lat_mesh.flatten(), lon_mesh.flatten()))
    values = z.flatten()
    
# create array of target points
    xi = np.column_stack((recon_lats, recon_lons))
    
     Interpolate elevation values at the target points
    elevation = griddata(points, values, xi, method='linear')
    
    return elevation

target_points = [
    (20.434572, 6.989361),
    (38.961984, 25.387083),
    (28.225464, 31.435073),
    (34.029520, 26.528961),
    (45.001242, 26.379404),
    (27.162020, 27.901794),
    (44.048547, 33.979269),
    (31.586694, 25.435194),
    (34.967497, 21.885426),
    (37.419805, 27.925387),
    (26.696380, 31.962176),
    (37.536909, 27.877383),
    (34.783539, 21.867301),
    (41.117205, 20.589335),
    (24.554648, 18.188033),
    (36.638260, 27.831164),
    (43.589623, 28.770044),
    (36.486303, 25.355519),
    (14.887676, 32.705544),
    (26.022205, 32.081131),
    (41.029868, 20.579030),
    (23.394308, 14.877934),
    (46.260636, 28.959611),
    (36.995575, 16.079762),
    (26.574802, 32.186236),
    (43.488719, 33.480240),
    (26.347982, 10.715752),
    (19.169121, 31.075515),
    (40.332396, 11.825718),
    (19.439008, 10.240827),
    (29.484933, 27.937313),
    (37.375931, 27.999062),
    (36.702567, 27.962623),
    (31.732089, 26.745355),
    (39.447970, 17.440738),
    (32.489450, 27.602206),
    (34.623117, 24.306109),
    (41.543382, 26.152684),
    (31.839077, 31.190986),
    (32.045752, 26.424513),
    (19.445471, 31.170446),
    (23.677075, 28.795486),
    (35.721142, 10.768188),
    (45.887304, 13.598371),
    (29.357023, 15.423959),
    (35.863229, 18.666081),
    (38.732988, 17.558699),
    (42.368080, 34.326134),
    (79.716302, -33.717361),
    (79.804895, -56.401371),
    (59.503112, 152.673563),
    (30.151131, 112.918703),
    (43.378243, 146.920745),
    (52.029301, -170.247947),
    (50.767596, -167.215695),
    (52.362826, 139.474821),
    (52.362826, 139.474821),
    (-45.890991, -21.386685),
    (3.869406, -35.624184),
    (20.329999, -35.999739),
    (32.534049, -39.460509),
    (32.534049, -39.460509),
    (-4.263128, -33.134764),
    (39.774456, -55.479838),
    (43.238519, 6.072109),
    (32.753371, -17.126329)
]

# split the points into lat and lon arrays
recon_lats = np.array([p[0] for p in target_points])
recon_lons = np.array([p[1] for p in target_points])

# extract elevations
elevations = find_elevation_from_nc(recon_lats, recon_lons, lat, lon, z)

# print results
print(f"{'Latitude':>12} {'Longitude':>12} {'Elevation':>12}")
print("-" * 60)

for point, elev in zip(target_points, elevations):
    print(f"{point[0]:12.4f} {point[1]:12.4f} {elev:12.2f}")


In [ ]:
# create raster file from elevation values from nc file 

lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
z = nc.variables['z'][:]

z_anox= 1*z
for i in np.arange (0, 181):
    for j in np.arange (0, 361):
        if z [i, j]< -2375 and z[i, j]> -3706: #change these variables 
            z_anox [i,j]= 1
        else: 
            z_anox [i, j]= 0

#plot contour 
plt.figure(figsize=(20,10))
plt.contourf(lon, lat, z_anox)
colorbar= plt.colorbar()
colorbar.set_label('Elevation (m)', fontsize= 14)
plt.gca().set_aspect("equal")

#for lon, lat, label in zip(data_lons, data_lats, labels):
 #   if label == 'Euxinic ':
  #      plt.scatter(lon, lat, color='maroon', marker='o') # Plot 'Oxic' points
   # elif label == 'Anoxic ':
    #    plt.scatter(lon, lat, color='lightcoral', marker='o')
  #  elif label == 'Anoxic':
   #     plt.scatter(lon, lat, color='lightcoral', marker='o')
    #elif label == 'Suboxic ':
     #   plt.scatter(lon, lat, color='darkorange', marker='o')
    #else:
     #  print('youre missing one')

print(z_anox)

In [ ]:
# grid box area in km2 and total global area
Re = 6371 ; #earth radius km
delta_lat = 1 ; #model resolution lat
delta_lon = 1 ; # model resolution lon
dist_lat =  2* np.pi * Re * delta_lat / 360 ;
dist_lon =  2* np.pi * Re * np.cos(np.deg2rad(lat)) * delta_lon/ 360 ;
gridarea_vec = dist_lon * dist_lat ;
gridarea = np.empty([181, 361]) ;

#print(gridarea_vec.shape)
#gridarea[:,0] = gridarea_vec
for n in np.arange(0, 360):
    gridarea[:,n] = gridarea_vec

plt.imshow(gridarea)
plt.colorbar()
print(np.sum(gridarea))

In [ ]:
# calculation of maximal estimate (everywhere at specific elevation)
anoxarea= np.sum(z_anox*gridarea)
worldarea= np.sum(gridarea)
anoxfrac= (anoxarea/worldarea)*100
print(anoxfrac)

In [ ]:
# convert raster to csv for manual editing in excel (for conservative estimate)

def raster_to_matrix_csv(z_anox, lat, lon, output_path):
    # create dataFrame with z_anox values
    # using latitude (rows) and longitude (columns) as index
    df = pd.DataFrame(
        data=z_anox,
        index=lat,
        columns=lon
    )
    
    # add latitude label to index
    df.index.name = 'Latitude'
    
    # export to CSV
    df.to_csv(output_path)
    print(f"CSV file saved to: {output_path}")
    
    # print shape information
    print("\nData Summary:")
    print(f"Matrix shape: {z_anox.shape}")
    print(f"Number of latitudes: {len(lat)}")
    print(f"Number of longitudes: {len(lon)}")

# save file to computer 
output_path = 'z_anox_matrix_toae_95.csv'
raster_to_matrix_csv(z_anox, lat, lon, output_path)

In [ ]:
# load back in edited csv file 
conservative_data = pd.read_csv("z_anox_matrix_toae_95_edited.csv", index_col=0)

In [ ]:
# get longitude and latitude values
lons = conservative_data.columns.astype(float)
lats = conservative_data.index.astype(float)

# create meshgrid for contourf
lon_mesh, lat_mesh = np.meshgrid(lons, lats)

# create the plot
plt.figure(figsize=(12, 8))

# plot using contourf using only 2 levels since data is binary
contour = plt.contourf(lon_mesh, lat_mesh, conservative_data, 
                      levels=[-0.5, 0.5, 1.5],  
                      colors=['purple', 'yellow'],  
                      extend='neither')  
plt.gca().set_aspect("equal")

# colourbar
cbar = plt.colorbar(contour, ticks=[0, 1])
cbar.set_label('Binary Value (0/1)')

# labels and title 
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Conservative estimate')

# gridlines
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()

In [ ]:
#define z and elevations (z) at which we expect anoxia 
z = nc.variables['z'][:]
data = np.where(conservative_data == 1, 1, 0)

In [ ]:
# calculation of conservative estimate (where datapoints have coincided with area at that elevation)
anoxarea= np.sum(data*gridarea)
worldarea= np.sum(gridarea)
anoxfrac= (anoxarea/worldarea)*100
print(anoxfrac)

In [ ]:
# creation of map of conservative estimate (can be edietd to create map of maximal estimate)

lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
z = nc.variables['z'][:]

# create figure and axis
plt.figure(figsize=(20,10))
plt.tight_layout()

# main contour plot
main_contour = plt.contourf(lon, lat, z, cmap=cmocean.cm.deep_r)
colorbar = plt.colorbar()
colorbar.set_label('Elevation (m)', fontsize=14)

# create contour lines of the area
#contour_lines = plt.contour(lon, lat, data, levels=[0], 
 #                          colors='lightcoral', linewidths=1.5, 
  #                         linestyles='dashed')

# add filled regions
lon = lon[1:-1]
lat = lat[::-1]
data_95_array = data[:, 1:-1]  # i removed the first and last columns using numpy slicing due to my data not fitting 
contourf_regions = plt.contourf(lon, lat, data_95_array, 
                               levels=[0.5, 1.5],  
                               colors=['lightcoral'],
                               alpha=0.5,
                               zorder=1)
plt.gca().set_aspect("equal")

# plot scatter points by oxygen state
for lon, lat, label in zip(data_lons, data_lats, labels):
    if label == 'Euxinic ':
        plt.scatter(lon, lat, color='maroon', marker='o') # Plot 'Oxic' points
    elif label == 'Anoxic ':
        plt.scatter(lon, lat, color='lightcoral', marker='o')
    elif label == 'Anoxic':
        plt.scatter(lon, lat, color='lightcoral', marker='o')
    elif label == 'Suboxic ':
        plt.scatter(lon, lat, color='darkorange', marker='o')
    else:
        print('youre missing one')

# labels and title
plt.xlabel('Longitude', fontsize=14)
plt.ylabel('Latitude', fontsize=14)
plt.title('Map of the Cenomanian/Turonian period with Sample Data Points and Maximal Estimate for the Extent of Anoxia', fontsize=16)

# legend
Anoxic_patch = mpatches.Patch(color='lightcoral', label='Anoxic')
Suboxic_patch = mpatches.Patch(color='darkorange', label='Suboxic')
Oxic_patch = mpatches.Patch(color='khaki', label='Oxic')
z_anox_line = mpatches.Patch(facecolor='lightcoral', label='Maximal Extent of Anoxia', alpha=0.5)
plt.legend(handles=[Anoxic_patch, Suboxic_patch, Oxic_patch, z_anox_line], loc='upper left')

plt.show()